In [1]:
import pandas as pd
from transformers import pipeline
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

In [2]:
rules_traffic="Speed limits: Every road or highway has a posted speed limit, and drivers are required to obey these limits. It's important to note that in some cases, there may be a minimum speed limit as well;Traffic signals, such as stop signs, red lights, and green lights, are used to regulate the flow of traffic and prevent accidents. Drivers are required to obey these signals at all times.;Right of way In situations where two or more vehicles are approaching an intersection or other traffic situations, the driver who has the right of way is the one who has the legal right to proceed first. It's important to understand and follow right-of-way laws to prevent accidents.;Seat belt laws: In most states, drivers and passengers are required to wear seat belts at all times while the vehicle is in motion. Failing to do so can result in fines or other penalties.;DUI laws: Driving under the influence of drugs or alcohol is illegal in all states, and the penalties for DUI can be severe. It's important to never drink and drive, and to always designate a sober driver or use alternative transportation if you plan on consuming alcohol.;Turning signals: Drivers are required to use their turn signals when turning or changing lanes;Pedestrian safety: Drivers must yield to pedestrians in crosswalks and other designated areas;Cell phone use while driving: In many states, it's illegal to use a cell phone while driving, unless it's a hands-free device.Headlight use: Headlights must be used during nighttime driving, and in certain weather conditions, such as fog or heavy rain"
loopholes_traffic="If a driver is experiencing a medical emergency and needs to get to the hospital quickly, they may argue that they had to exceed the speed limit in order to get medical help in time. However, this would only be a valid defense if the emergency was real and required immediate attention;Traffic signals: If a driver can argue that the traffic signal was malfunctioning or that they were forced to run a red light to avoid an accident, they may be able to avoid a ticket. However, it would be difficult to prove that the traffic signal was actually malfunctioning.;Right of way: If a driver can argue that they were not aware of the right-of-way rules or that the other driver was not following them correctly, they may be able to avoid liability. However, it's important to always follow the right-of-way rules and be aware of other drivers on the road.;Seat belt laws: If a driver can prove that they were not wearing a seat belt due to a medical condition or disability, they may be able to avoid a fine. However, it's important to always wear a seat belt for your safety and the safety of others.;DUI laws: There are no legitimate loopholes to DUI laws. Driving under the influence of drugs or alcohol is illegal and can have serious consequences. It's important to never drink and drive, and to always designate a sober driver or use alternative transportation if you plan on consuming alcohol.;However, some drivers may neglect to use their signals, especially when driving in heavy traffic or in an emergency situation. They may argue that using their signals would have been unsafe or that they didn't have enough time to signal before making the turn.;However, some drivers may attempt to find loopholes by claiming that the pedestrian was not following proper pedestrian safety rules or by arguing that the pedestrian was not visible due to poor lighting or weather conditions;Some drivers may attempt to find loopholes by using their phone for emergency purposes or by claiming that they were not actively using their phone while driving.;drivers may try to find loopholes by not turning on their headlights in certain situations, such as driving in a well-lit area or claiming that their headlights were not functioning properly."



In [3]:
loopholes_buisness="Payment terms: A company could set very short payment terms, such as net-10, in order to put pressure on customers to pay quickly. This could be unfair to customers who are unable to pay within such a short time frame.;Return policies: A company could have a very restrictive return policy, such as a no-return policy, in order to discourage customers from returning products. This could be unfair to customers who are unhappy with a product and want to return it.;Employee benefits: A company could offer very limited employee benefits, such as no health insurance or retirement plan, in order to save money. This could be unfair to employees who rely on these benefits.;Quality control: A company could have very low quality control standards, in order to save money. This could lead to products that are not of good quality, which could harm the company's reputation.;Marketing promotions: A company could offer marketing promotions that are only available to a select group of customers, such as those who sign up for a mailing list. This could be unfair to customers who are not aware of the promotion or who are not eligible to participate.;Factories Act, 1948:;The Factories Act does not cover all workplaces in India. For example, it does not cover workplaces with less than 10 workers.;The Factories Act is not always enforced effectively. This is due to a number of factors, including lack of resources and corruption.;Shops and Commercial Establishments Acts:;The Shops and Commercial Establishments Acts vary from state to state. This can make it difficult for businesses to comply with the law.The Shops and Commercial Establishments Acts are not always enforced effectively. This is due to a number of factors, including lack of resources and corruption.;The Contract Labour (Regulation and Abolition) Act, 1970:The Contract Labour (Regulation and Abolition) Act is difficult to enforce. This is due to a number of factors, including the fact that it is often difficult to identify whether workers are employed as contract labour or as regular employees.;The Minimum Wages Act, 1948:The Minimum Wages Act is not always enforced effectively. This is due to a number of factors, including lack of resources and corruption.;The Payment of Wages Act, 1936:The Payment of Wages Act is not always enforced effectively. This is due to a number of factors, including lack of resources and corruption.;The Payment of Bonus Act, 1965:The Payment of Bonus Act is not always enforced effectively. This is due to a number of factors, including lack of resources and corruption.;The Maternity Benefit Act, 1961:The Maternity Benefit Act is not always enforced effectively. This is due to a number of factors, including lack of resources and corruption.;The Equal Remuneration Act, 1976:The Equal Remuneration Act is not always enforced effectively. This is due to a number of factors, including lack of resources and corruption.;The Workmen's Compensation Act, 1923:The Workmen's Compensation Act is not always enforced effectively. This is due to a number of factors, including lack of resources and corruption.Companies Act, 2013:There is no provision for mandatory rotation of auditors. This can lead to auditors becoming too close to the companies they audit, and may make them less likely to raise concerns about potential wrongdoing.The law does not require companies to have a whistleblowing policy in place. This can make it difficult for employees to report wrongdoing without fear of retaliation.;Limited Liability Partnership Act, 2008:There is no requirement for limited liability partnerships to have a minimum number of members. This could lead to the formation of LLPs with only a few members, which could make it easier for the members to engage in fraudulent or other illegal activities.;Partnership Act, 1932:There is no requirement for partnerships to be registered. This could make it difficult for creditors to track down partners who owe them money.;Contract Act, 1872:The law does not define the term consideration. This could lead to disputes about whether a contract is valid if the parties do not agree on what constitutes consideration.;Sale of Goods Act, 1930:The law does not require sellers to provide a warranty with the goods they sell. This could leave buyers with no recourse if the goods they purchase are defective.;Consumer Protection Act, 1986:The law does not set a limit on the amount of compensation that can be awarded to consumers who have been harmed by defective goods or services. This could discourage consumers from filing claims, as they may not be able to recover the full amount of their losses.;Intellectual Property Rights Act, 1999:The law does not provide for criminal penalties for infringement of intellectual property rights. This could make it difficult to deter infringement, as infringers may not face serious consequences for their actions.;Foreign Exchange Management Act, 1999:The law does not provide for a mechanism for individuals to appeal decisions made by the Reserve Bank of India (RBI) under the Act. This could lead to individuals being unfairly denied access to foreign exchange, as they may not have a way to challenge the RBI's decision.;Competition Act, 2002:The law does not provide for a private right of action for individuals who have been harmed by anti-competitive practices. This could discourage individuals from taking legal action against companies that engage in anti-competitive behavior, as they may not be able to recover their losses."
rules_buisness="Payment terms: A business rule could be the payment terms that a company sets for its customers, such as net-30 or net-60. This rule could specify the amount of time a customer has to pay for a product or service, and any consequences for late payment.;Return policies: Another business rule could be the return policy that a company sets for its products. This rule could specify the conditions under which a customer can return a product, such as a time frame or condition of the product, and any associated fees or penalties.;Employee benefits: A business rule could govern the benefits that a company offers to its employees, such as health insurance or retirement plans. This rule could specify the eligibility requirements for these benefits, the types of benefits offered, and any associated costs or contributions.;Quality control: A business rule could establish quality control standards for a company's products or services. This rule could specify the procedures or guidelines that employees must follow to ensure consistent quality, and any consequences for failing to meet these standards.;Marketing promotions: A business rule could govern the marketing promotions that a company offers to its customers, such as discounts or coupons. This rule could specify the terms of the promotion, such as the duration, eligibility requirements, and any associated restrictions or limitations.Factories Act, 1948: This Act regulates the working conditions in factories in India. The Factories Act sets out the minimum standards for health, safety, and welfare of workers in factories.;Shops and Commercial Establishments Acts: These Acts regulate the working hours and conditions of employment in shops and commercial establishments in India. The Shops and Commercial Establishments Acts vary from state to state.;The Contract Labour (Regulation and Abolition) Act, 1970: This Act regulates the employment of contract labour in India. The Contract Labour (Regulation and Abolition) Act prohibits the employment of contract labour in certain industries and occupations. It also requires that contract labour be employed only when it is necessary and in accordance with the Act.;The Minimum Wages Act, 1948: This Act fixes the minimum wages for workers in India. The Minimum Wages Act sets out the minimum wages for different types of work and occupations. It also provides for the enforcement of the minimum wages.;The Payment of Wages Act, 1936: This Act regulates the payment of wages to workers in India. The Payment of Wages Act requires that wages be paid in cash and on a regular basis. It also provides for the payment of overtime wages and for the recovery of wages due to workers.;The Payment of Bonus Act, 1965: This Act regulates the payment of bonus to workers in India. The Payment of Bonus Act requires that bonus be paid to workers in certain industries and occupations. It also sets out the formula for calculating bonus.;The Maternity Benefit Act, 1961: This Act provides for maternity benefits to women workers in India. The Maternity Benefit Act entitles women workers to paid maternity leave, crèche facilities, and other benefits.;The Equal Remuneration Act, 1976: This Act prohibits discrimination in remuneration between men and women workers in India. The Equal Remuneration Act prohibits employers from paying women workers less than men workers for the same work.;The Workmen's Compensation Act, 1923: This Act provides for compensation to workers who are injured or killed in the course of employment in India. The Workmen's Compensation Act requires employers to insure their workers against accidents and occupational diseases. It also provides for the payment of compensation to workers who are injured or killed in the course of employment."

In [4]:
loopholes_property="The Act does not define what constitutes movable and immovable property. This could be used to argue that certain types of property, such as intellectual property, are not covered by the Act.The Act does not explicitly state that property can only be transferred by a registered deed. This could be used to argue that property can be transferred by other means, such as by oral agreement.;Indian Easement Act, 1882:The Act does not define what constitutes an easement. This could be used to argue that certain types of rights, such as the right to use a neighbor's driveway, are not easements and are therefore not protected by the Act.The Act does not specify how easements can be created or terminated. This could be used to argue that easements can be created or terminated in ways that are not specified in the Act.;Registration Act, 1908The Act does not require that all documents relating to the transfer of property be registered. This could be used to argue that certain types of documents, such as informal agreements, are not required to be registered and are therefore not subject to the Act's provisions.The Act does not specify the consequences of failing to register a document. This could be used to argue that failing to register a document does not invalidate the document or the transaction that it evidences.;Real Estate (Regulation and Development) Act, 2016 (RERAThe Act does not apply to all real estate projects. This could be used to argue that developers of projects that are not subject to RERA are not required to comply with the Act's provisions.The Act does not specify the consequences of a developer failing to comply with RERA's provisions. This could be used to argue that failing to comply with RERA does not invalidate a contract between a developer and a homebuyer.;Right to Fair Compensation and Transparency in Land Acquisition, Rehabilitation and Resettlement Act, 2013:The Act does not specify the amount of compensation that must be paid to landowners when their land is acquired by the government. This could be used to argue that the government is not required to pay fair compensation to landowners.The Act does not specify the standards that must be met by the government's rehabilitation and resettlement program. This could be used to argue that the government's rehabilitation and resettlement program is not adequate.;The Indian Contract Act, 1872:The Act does not specify the requirements for a contract to be valid. This could be used to argue that certain types of contracts, such as contracts that are not in writing, are not valid.The Act does not specify the remedies that are available to a party who is the victim of a breach of contract. This could be used to argue that a party who is the victim of a breach of contract is not entitled to any remedies.;The Specific Relief Act, 1963:The Act does not specify the conditions that must be met in order for a court to grant specific performance or an injunction. This could be used to argue that a court is not required to grant specific performance or an injunction in any particular case.;The Law of Torts:The law of torts is not codified in India. This means that there is no single statute that defines the torts that are recognized by Indian law. This could be used to argue that certain types of conduct, such as negligence, are not torts and are therefore not subject to the law of torts.;The Environmental Protection Act, 1986:The Act does not specify the penalties that can be imposed for violations of the Act. This could be used to argue that the penalties that are imposed for violations of the Act are not adequate.;The Benami Transactions (Prohibition) Act, 1988:The Act does not define what constitutes a benami transaction. This could be used to argue that certain types of transactions are not benami transactions and are therefore not prohibited by the Act."
rule_property="Transfer of Property Act, 1882: This Act governs the transfer of movable and immovable property. It deals with the different types of property, the different ways in which property can be transferred, and the rights and obligations of the parties involved in a property transaction.;Indian Easement Act, 1882: This Act deals with easements, which are rights that one person has over the property of another. For example, a person may have the right to use a neighbor's driveway to access their own property.;Registration Act, 1908: This Act requires that certain documents, including documents relating to the transfer of property, be registered with the government. Registration provides a public record of the ownership of property and helps to prevent fraud.;Real Estate (Regulation and Development) Act, 2016 (RERA): This Act was enacted to protect the interests of homebuyers in India. RERA requires developers to register their projects with the government and to comply with certain regulations, such as providing timely updates to homebuyers and completing projects on time.;Right to Fair Compensation and Transparency in Land Acquisition, Rehabilitation and Resettlement Act, 2013: This Act was enacted to ensure that fair compensation is paid to landowners when their land is acquired by the government for public purposes. The Act also requires the government to provide rehabilitation and resettlement assistance to affected families.;The Indian Contract Act, 1872: This Act governs all contracts, including contracts for the sale of property. It deals with the formation of contracts, the rights and obligations of the parties to a contract, and the remedies available for breach of contract.;The Specific Relief Act, 1963: This Act provides for specific reliefs, such as specific performance and injunctions, that can be granted by the courts in certain cases. Specific performance is an order of the court that requires a party to a contract to perform their obligations under the contract. An injunction is an order of the court that prohibits a party from doing something.;The Law of Torts: Torts are civil wrongs that are not based on contract. Some examples of torts include negligence, trespass, and defamation. The law of torts provides for remedies, such as damages, that can be awarded to the victim of a tort.;The Environmental Protection Act, 1986: This Act is designed to protect the environment from pollution and other forms of degradation. It gives the government the power to regulate activities that may harm the environment, and it provides for penalties for violations of the Act.;The Benami Transactions (Prohibition) Act, 1988: This Act prohibits benami transactions, which are transactions in which property is held in the name of a person who is not the real owner. The Act was enacted to prevent money laundering and other forms of financial crime."

In [5]:
loopholes_tax="Gift tax loopholes. There are a number of loopholes that can be used to avoid paying gift tax. For example, individuals can give gifts up to a certain amount each year without paying any gift tax. Additionally, individuals can give gifts to their spouse without paying any gift tax.;Inheritance tax loopholes. There are also a number of loopholes that can be used to avoid paying inheritance tax. For example, individuals can leave their property to their spouse or to a charitable organization without paying any inheritance tax. Additionally, individuals can leave their property to trusts that are designed to avoid inheritance tax.;Corporate tax loopholes. Corporations can use a number of loopholes to avoid paying corporate tax. For example, corporations can move their profits to tax havens or they can use complex accounting methods to reduce their taxable income.;Sales tax loopholes. There are also a number of loopholes that can be used to avoid paying sales tax. For example, individuals can purchase goods or services online and avoid paying sales tax. Additionally, individuals can purchase goods or services from businesses that are located in states that do not have sales tax.;Customs duty loopholes. Customs duty is a tax that is levied on the import or export of goods. There are a number of loopholes that can be used to avoid paying customs duty. For example, businesses can miss declare the value of goods that they are importing or exporting. Additionally, businesses can use complex shipping methods to avoid paying customs duty.;Excise duty loopholes. Excise duty is a tax that is levied on the production or sale of goods within a country. There are a number of loopholes that can be used to avoid paying excise duty. For example, businesses can misdeclare the quantity of goods that they are producing or selling. Additionally, businesses can use complex manufacturing methods to minimize their excise duty liability.;Income tax: One of the most common loopholes in income tax is the use of shell companies. Shell companies are companies that are set up with the sole purpose of avoiding taxes. They are often used to park funds that are earned illegally or to claim deductions that are not actually available.;Goods and services tax (GST): One of the most common loopholes in GST is the use of fake invoices. Fake invoices are invoices that are created for goods or services that were never actually supplied. They are often used to claim input tax credit that is not actually due.;Wealth tax: One of the most common loopholes in wealth tax is the undervaluation of assets. Assets are often undervalued in order to reduce the amount of wealth tax that is payable. This can be done by artificially inflating the cost of liabilities or by claiming that assets are not as valuable as they actually are.;Estate duty: One of the most common loopholes in estate duty is the use of trusts. Trusts are legal entities that are created to hold assets for the benefit of another person. They are often used to avoid estate duty by transferring assets to a trust before death.;Customs duty: One of the most common loopholes in customs duty is the undervaluation of goods. Goods are often undervalued in order to reduce the amount of customs duty that is payable. This can be done by artificially inflating the cost of freight or by claiming that goods are not as valuable as they actually are."
rules_tax="Inheritance tax is a tax that is levied on the transfer of property by one person to another after their death. The inheritance tax is payable by the person who inherits the property.;Corporate tax. Corporate tax is a tax that is levied on the profits of corporations. The corporate tax rate varies depending on the size of the corporation and the amount of profit that is made. ;Sale Sales tax is a tax that is levied on the sale of goods and services. The sales tax rate varies depending on the type of goods or services that are sold.;Excise duty. Excise duty is a tax that is levied on the production or sale of goods within a country. The excise duty rate varies depending on the type of goods that are produced or sold.;Income tax: The Income-tax Act, 1961, provides for the levy and collection of income tax on individuals, companies, and other entities. The Act also provides for the rates of income tax, the deductions and exemptions that are available, and the procedures for filing income tax returns.;Goods and services tax (GST): The Central Goods and Services Tax Act, 2017, provides for the levy and collection of goods and services tax (GST) on goods and services supplied in India. The Act also provides for the rates of GST, the exemptions that are available, and the procedures for filing GST returns.;Wealth tax: The Wealth-tax Act, 1957, provides for the levy and collection of wealth tax on individuals and Hindu undivided families. The Act also provides for the rates of wealth tax, the exemptions that are available, and the procedures for filing wealth tax returns.;Gift tax: The Gift-tax Act, 1958, provides for the levy and collection of gift tax on gifts received by individuals and Hindu undivided families. The Act also provides for the rates of gift tax, the exemptions that are available, and the procedures for filing gift tax returns.;Estate duty: The Estate Duty Act, 1953, provides for the levy and collection of estate duty on the transfer of property on death. The Act also provides for the rates of estate duty, the exemptions that are available, and the procedures for filing estate duty returns.;Customs duty: The Customs Act, 1962, provides for the levy and collection of customs duty on goods imported into or exported from India. The Act also provides for the rates of customs duty, the exemptions that are available, and the procedures for filing customs duty returns.."

In [6]:
rules=rules_traffic+rules_buisness+rule_property+rules_tax
loopholes=loopholes_traffic+loopholes_buisness+loopholes_property+loopholes_tax

In [7]:
total=rules+loopholes
rule_list=total.split(';')
print(rule_list)

["Speed limits: Every road or highway has a posted speed limit, and drivers are required to obey these limits. It's important to note that in some cases, there may be a minimum speed limit as well", 'Traffic signals, such as stop signs, red lights, and green lights, are used to regulate the flow of traffic and prevent accidents. Drivers are required to obey these signals at all times.', "Right of way In situations where two or more vehicles are approaching an intersection or other traffic situations, the driver who has the right of way is the one who has the legal right to proceed first. It's important to understand and follow right-of-way laws to prevent accidents.", 'Seat belt laws: In most states, drivers and passengers are required to wear seat belts at all times while the vehicle is in motion. Failing to do so can result in fines or other penalties.', "DUI laws: Driving under the influence of drugs or alcohol is illegal in all states, and the penalties for DUI can be severe. It's 

In [8]:
df = pd.DataFrame({'rules': rule_list})
pd.set_option('display.max_rows', None)
print(df)

                                                rules
0   Speed limits: Every road or highway has a post...
1   Traffic signals, such as stop signs, red light...
2   Right of way In situations where two or more v...
3   Seat belt laws: In most states, drivers and pa...
4   DUI laws: Driving under the influence of drugs...
5   Turning signals: Drivers are required to use t...
6   Pedestrian safety: Drivers must yield to pedes...
7   Cell phone use while driving: In many states, ...
8   Return policies: Another business rule could b...
9   Employee benefits: A business rule could gover...
10  Quality control: A business rule could establi...
11  Marketing promotions: A business rule could go...
12  Shops and Commercial Establishments Acts: Thes...
13  The Contract Labour (Regulation and Abolition)...
14  The Minimum Wages Act, 1948: This Act fixes th...
15  The Payment of Wages Act, 1936: This Act regul...
16  The Payment of Bonus Act, 1965: This Act regul...
17  The Maternity Benefit Ac

In [9]:
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def tokenize_text(text):
    tokens = tokenizer.tokenize(text)
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    return tokens


In [11]:
df['tokens'] = df['rules'].apply(tokenize_text)
def convert_tokens_to_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

In [12]:
df['token_ids'] = df['tokens'].apply(convert_tokens_to_ids)
MAX_LENGTH = 128

In [13]:
def pad_sequences(token_ids):
    if len(token_ids) > MAX_LENGTH:
        token_ids = token_ids[:MAX_LENGTH]
    else:
        token_ids = token_ids + [0] * (MAX_LENGTH - len(token_ids))
    return token_ids

In [14]:
df['padded_token_ids'] = df['token_ids'].apply(pad_sequences)
def prepare_input_tensors(padded_token_ids):
    input_ids = torch.tensor([padded_token_ids])
    return input_ids


In [15]:
df['input_tensors'] = df['padded_token_ids'].apply(prepare_input_tensors)
def extract_bert_embeddings(input_tensors):
    with torch.no_grad():
        model.eval()
        outputs = model(input_tensors)
        embeddings = outputs[0][0]
    return embeddings


In [16]:
df['embeddings'] = df['input_tensors'].apply(extract_bert_embeddings)

print("Original Text:", df['rules'][0])
print("Tokenized Text:", df['tokens'][0])
print("Token IDs:", df['token_ids'][0])
print("Padded Token IDs:", df['padded_token_ids'][0])
print("Input Tensors:\n", df['input_tensors'][0])
print("BERT Embeddings:\n", df['embeddings'][0])

Original Text: Speed limits: Every road or highway has a posted speed limit, and drivers are required to obey these limits. It's important to note that in some cases, there may be a minimum speed limit as well
Tokenized Text: ['[CLS]', 'speed', 'limits', ':', 'every', 'road', 'or', 'highway', 'has', 'a', 'posted', 'speed', 'limit', ',', 'and', 'drivers', 'are', 'required', 'to', 'obey', 'these', 'limits', '.', 'it', "'", 's', 'important', 'to', 'note', 'that', 'in', 'some', 'cases', ',', 'there', 'may', 'be', 'a', 'minimum', 'speed', 'limit', 'as', 'well', '[SEP]']
Token IDs: [101, 3177, 6537, 1024, 2296, 2346, 2030, 3307, 2038, 1037, 6866, 3177, 5787, 1010, 1998, 6853, 2024, 3223, 2000, 15470, 2122, 6537, 1012, 2009, 1005, 1055, 2590, 2000, 3602, 2008, 1999, 2070, 3572, 1010, 2045, 2089, 2022, 1037, 6263, 3177, 5787, 2004, 2092, 102]
Padded Token IDs: [101, 3177, 6537, 1024, 2296, 2346, 2030, 3307, 2038, 1037, 6866, 3177, 5787, 1010, 1998, 6853, 2024, 3223, 2000, 15470, 2122, 6537, 10